In [4]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [5]:
#parameters
scan_id=3
sample_rate=10
no_of_lines =50
line_tolarance=1.5
model_name="svm"

In [6]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [7]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()

In [8]:
def project_points(point_array, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    if(axis==0):
        point_array[:,0]=0
    if(axis==1):
        point_array[:,1]=0
    if(axis==2):
        point_array[:,2]=0
        
    return point_array

In [9]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        
        if(len(candidates)==0 or len(outline_points)==no_of_lines):
            break
            
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        
        outline_points.append(line_points)
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [10]:
def read_dataset():
    all_set = pd.read_csv("Datasets/all_data.csv", sep=',',header=0)
    train_set=pd.read_csv("Datasets/train_set.csv", sep=',',header=0)
    test_set=pd.read_csv("Datasets/test_set.csv", sep=',',header=0)
    
    return all_set.drop(all_set.index[[0,1]]),train_set.drop(train_set.index[[0,1]]), test_set.drop(test_set.index[[0,1]])

In [11]:
def load_model(model=model_name):
    if(model=="svm"):
        return pickle.load(open("Models/SVM_Classifier.pkcls", 'rb'))
    else:
        return

In [109]:
points_0 = open("Datasets/Foot Models/points/left/3/sub103ll.pts")
point_array_0 = process_data(points_0)
projection_0 = project_points(point_array_0)
outline_0 =get_outline(projection_0)
outline_points_0= outline_0[0]
lines_0=outline_0[1]

In [110]:
plot_graph(projection_0, outline_points=outline_points_0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
dataset= read_dataset()
all_data= dataset[0]
train_set = dataset[1]
test_set = dataset[2]

In [112]:
model=load_model() #implement classification model
test_features=test_set[["No.","Cluster","Foot Length","Foot Breadth 1","Short Heel Girth"]]

In [104]:
c4_feet = np.array(train_set.loc[train_set["Cluster"]=='C4']["No."])

c4_outline_points = [] 

for foot in c4_feet:
    points = open("Datasets/Foot Models/points/left/3/"+foot+"3ll.pts")
    point_array = process_data(points)
    projection = project_points(point_array)
    outline =get_outline(projection)
    outline_points= outline[0]
    lines=outline[1]
    
    c4_outline_points.append(outline_points)

In [137]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function to minimize
def objective(x):
    func = 0
    for i in range(0,no_of_lines):
        for j in range(0,2):
            weighted_point = 0
            for k in range(0,len(c4_outline_points)):
                weighted_point = weighted_point + x[k]*c4_outline_points[k][i][j] 
            func = func+ np.linalg.norm(weighted_point-outline_points_0[i][j])
    return func

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        print("Processing Gen:",gen)
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
                
#                 val=0
#                 for j in range(len(decoded[i])):
#                     val=val+c4_outline_points[j]*decoded[i][j]
#                 print(val.shape)
#                 plot_graph(projection_0, outline_points=val)
                    
                
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define range for input
bounds = [[[0,2]]*len(c4_outline_points)][0] #negative range will not provide smooth shape. only a scaling a factor should be determined
# define the total iterations
n_iter = 1000 # take around 1 hour for 5000 iterations for cluster size of 34
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))


#set bound to (-1 to 1) and normalize the foots to max length foot

Processing Gen: 0
>0, new best f([0.41180419921875, 0.53369140625, 0.686187744140625, 1.462066650390625, 0.381072998046875, 0.5482177734375, 0.147613525390625, 1.451446533203125, 1.361053466796875, 0.4320068359375, 0.150115966796875, 0.6236572265625, 1.2138671875, 0.20379638671875, 0.54400634765625, 0.209259033203125, 1.316802978515625, 1.66265869140625, 0.7957763671875, 1.05914306640625, 1.704254150390625, 0.221588134765625, 1.3502197265625, 1.541107177734375, 0.6473388671875, 0.593292236328125, 1.265350341796875, 0.84112548828125, 0.7698974609375, 0.11004638671875, 0.8302001953125, 0.918426513671875, 1.84918212890625, 0.80682373046875]) = 642958.706801
>0, new best f([0.17279052734375, 0.060821533203125, 0.084503173828125, 1.28369140625, 0.91259765625, 0.96954345703125, 0.949432373046875, 0.884063720703125, 0.517822265625, 1.504669189453125, 0.509735107421875, 1.036163330078125, 1.48236083984375, 0.129638671875, 0.721923828125, 1.063262939453125, 1.320709228515625, 1.407470703125, 0.

In [138]:
decoded

[0.001007080078125,
 3.0517578125e-05,
 0.022918701171875,
 0.02532958984375,
 0.016998291015625,
 0.0001220703125,
 0.042633056640625,
 0.005523681640625,
 0.014404296875,
 0.049713134765625,
 0.02923583984375,
 0.12481689453125,
 0.0009765625,
 0.078094482421875,
 0.0159912109375,
 0.023101806640625,
 0.0087890625,
 0.089813232421875,
 0.07977294921875,
 0.073699951171875,
 0.058349609375,
 0.05572509765625,
 3.0517578125e-05,
 0.000396728515625,
 0.0130615234375,
 0.000244140625,
 0.01165771484375,
 0.0155029296875,
 0.04095458984375,
 0.046844482421875,
 0.0,
 0.001007080078125,
 0.00390625,
 0.06005859375]

In [139]:
val=0
for i in range(len(decoded)):
    val=val+c4_outline_points[i]*decoded[i]

In [140]:
plot_graph(projection_0, outline_points=val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
#todo project from specific height not all the points -> modify project points

(50, 2, 3)